In [34]:
import pandas as pd

In [60]:
# Read the data from the excel file
bridge = pd.read_excel('infrastructure/original/BMMS_overview.xlsx')
bridge.head(3)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise


In [61]:
bridge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21407 entries, 0 to 21406
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   road              21407 non-null  object 
 1   km                21406 non-null  float64
 2   type              21407 non-null  object 
 3   LRPName           21407 non-null  object 
 4   name              21060 non-null  object 
 5   length            21398 non-null  float64
 6   condition         21407 non-null  object 
 7   structureNr       21407 non-null  int64  
 8   roadName          21406 non-null  object 
 9   chainage          21407 non-null  object 
 10  width             18290 non-null  float64
 11  constructionYear  18289 non-null  float64
 12  spans             18290 non-null  float64
 13  zone              21406 non-null  object 
 14  circle            21406 non-null  object 
 15  division          21406 non-null  object 
 16  sub-division      21406 non-null  object

2.2 Missing coordinates for bridges and duplicate entries.

In [191]:
# delete the rows with missing values in column 'lat' or 'lon'
bridge_new = bridge.dropna(subset=['lat', 'lon'])

# delete the rows with 0 in column 'lat' or 'lon'
bridge_new = bridge_new[(bridge_new['lat'] != 0) & (bridge_new['lon'] != 0)]
bridge_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21296 entries, 0 to 21406
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   road              21296 non-null  object 
 1   km                21296 non-null  float64
 2   type              21296 non-null  object 
 3   LRPName           21296 non-null  object 
 4   name              20972 non-null  object 
 5   length            21287 non-null  float64
 6   condition         21296 non-null  object 
 7   structureNr       21296 non-null  int64  
 8   roadName          21295 non-null  object 
 9   chainage          21296 non-null  object 
 10  width             18290 non-null  float64
 11  constructionYear  18289 non-null  float64
 12  spans             18290 non-null  float64
 13  zone              21295 non-null  object 
 14  circle            21295 non-null  object 
 15  division          21295 non-null  object 
 16  sub-division      21295 non-null  object 
 17

In [192]:
# create a new column 'road_LRPName' by combining 'road' and 'LRPName'
bridge_new = bridge_new.copy()
bridge_new['road_LRPName'] = bridge_new['road'] + '_' + bridge_new['LRPName']

# add a column 'null_num' with the calculattion of the number of null values for the each row
bridge_new['null_num'] = bridge_new.isnull().sum(axis=1)
bridge_new.head(3)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,road_LRPName,null_num
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,...,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1,N1_LRP001a,0
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,...,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1,N1_LRP004b,0
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,...,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise,N1_LRP008b,3


In [193]:
# delete the rows with duplicated 'road_LRPName' and keep the row with the minimum number of null values
bridge_new = bridge_new.loc[bridge_new.groupby('road_LRPName')['null_num'].idxmin()]

# find the duplicated 'road_LRPName'
bridge_new[bridge_new.duplicated(subset='road_LRPName', keep=False)]

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,road_LRPName,null_num


In [195]:
# sort the data by 'road'
bridge_new = bridge_new.sort_values(by=['road', 'LRPName'])
bridge_new.head(3)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,road_LRPName,null_num
0,N1,1.800,Box Culvert,LRP001a,.,11.3,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,...,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1,N1_LRP001a,0
1,N1,4.925,Box Culvert,LRP004b,.,6.6,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,...,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1,N1_LRP004b,0
13420,N1,8.976,PC Girder Bridge,LRP008b,KANCHPUR PC GIRDER BRIDGE,397.0,C,101102,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,...,8.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702083,90.515917,bcs1,N1_LRP008b,0


In [196]:
bridge_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18323 entries, 0 to 15393
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   road              18323 non-null  object 
 1   km                18323 non-null  float64
 2   type              18323 non-null  object 
 3   LRPName           18323 non-null  object 
 4   name              18304 non-null  object 
 5   length            18323 non-null  float64
 6   condition         18323 non-null  object 
 7   structureNr       18323 non-null  int64  
 8   roadName          18323 non-null  object 
 9   chainage          18323 non-null  object 
 10  width             18182 non-null  float64
 11  constructionYear  18182 non-null  float64
 12  spans             18182 non-null  float64
 13  zone              18323 non-null  object 
 14  circle            18323 non-null  object 
 15  division          18323 non-null  object 
 16  sub-division      18323 non-null  object 
 17

2.3 Bridges located outside the country boundaries.

In [197]:
# Define latitude and longitude boundaries
LAT_MIN, LAT_MAX = 20, 28
LON_MIN, LON_MAX = 88, 93

In [198]:
# Find the rows with 'lat' and 'lon' out of bounds
bridge_new[(bridge_new['lat'] < LAT_MIN) | (bridge_new['lat'] > LAT_MAX) | (bridge_new['lon'] < LON_MIN) | (bridge_new['lon'] > LON_MAX)]

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,road_LRPName,null_num
16183,R241,26.292,Box Culvert,LRP026a,EKOR CHOL BOX CULVERT,12.0,B,114995,Auskandi-Raniganj-Jagannathpur-Pagla Road,26.292,...,5.0,Sylhet,Sylhet,Sunamganj,Chattak,91.544194,24.773694,bcs1,R241_LRP026a,0
20460,R241,27.743,Box Culvert,LRP027a,EKOR CHAI BOX CULVERT,19.9,D,114994,Auskandi-Raniganj-Jagannathpur-Pagla Road,27.743,...,3.0,Sylhet,Sylhet,Sunamganj,Chattak,91.542389,24.786833,bcs1,R241_LRP027a,0
17247,R241,28.211,Box Culvert,LRP028a,HIJLA BOX CULVERT,9.2,C,114993,Auskandi-Raniganj-Jagannathpur-Pagla Road,28.211,...,2.0,Sylhet,Sylhet,Sunamganj,Chattak,91.541778,24.790944,bcs1,R241_LRP028a,0
16185,R241,28.857,Baily with Steel Deck,LRP028c,MAJIDPUR TRUSS BRIDGE,37.9,B,114992,Auskandi-Raniganj-Jagannathpur-Pagla Road,28.857,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.542083,24.796694,bcs1,R241_LRP028c,0
2864,R241,29.840,Box Culvert,LRP029a,MAJIDEPUR BOX CULVERT,3.0,A,114991,Auskandi-Raniganj-Jagannathpur-Pagla Road,29.84,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.543889,24.805111,bcs1,R241_LRP029a,0
2865,R241,31.212,Baily with Steel Deck,LRP031a,KHASIRA BAILEY WITH STEEL DECK,17.5,A,114990,Auskandi-Raniganj-Jagannathpur-Pagla Road,31.212,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.538722,24.816556,bcs1,R241_LRP031a,0
17248,R241,32.128,Box Culvert,LRP032a,KHASIRA BOX CULVERT,6.0,C,114989,Auskandi-Raniganj-Jagannathpur-Pagla Road,32.128,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.534361,24.823833,bcs1,R241_LRP032a,0
2866,R241,32.713,Box Culvert,LRP032b,KOLKALI BOX CULVERT,3.0,A,114988,Auskandi-Raniganj-Jagannathpur-Pagla Road,32.713,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.530556,24.828056,bcs1,R241_LRP032b,0
2867,R241,33.126,Box Culvert,LRP033a,KOKAL BOX CULVERT,1.0,A,114987,Auskandi-Raniganj-Jagannathpur-Pagla Road,33.126,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.527917,24.830944,bcs1,R241_LRP033a,0
17249,R241,33.759,Baily with Steel Deck,LRP033b,GONGUR BAILEY WITH STEEL DECK,14.2,C,114986,Auskandi-Raniganj-Jagannathpur-Pagla Road,33.759,...,1.0,Sylhet,Sylhet,Sunamganj,Chattak,91.524083,24.835222,bcs1,R241_LRP033b,0


In [199]:
# Swap 'lat' and 'lon' if both are out of bounds
def swap_coordinates(row):
    if not (LAT_MIN <= row['lat'] <= LAT_MAX) and not (LON_MIN <= row['lon'] <= LON_MAX):
        row['lat'], row['lon'] = row['lon'], row['lat']  # Swap values
    return row

bridge_new = bridge_new.apply(swap_coordinates, axis=1)

In [200]:
# Find the rows with 'lat' and 'lon' out of bounds
bridge_new[(bridge_new['lat'] < LAT_MIN) | (bridge_new['lat'] > LAT_MAX) | (bridge_new['lon'] < LON_MIN) | (bridge_new['lon'] > LON_MAX)]

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,road_LRPName,null_num


In [201]:
# write the cleaned data to a new excel file
bridge_new.to_excel('sourcecode/BMMS_overview_cleaned_1.xlsx', index=False)   
